In [11]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core.tools import FunctionTool
from autogen_agentchat.agents import AssistantAgent,UserProxyAgent
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import asyncio
from pydantic import BaseModel,Field
from autogen_ext.tools.http import HttpTool
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.base import TaskResult
from autogen_agentchat.ui import Console



# Create the token provider
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default",
)

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    model="gpt-4o-mini",
    api_version="2024-08-01-preview",
    azure_endpoint="https://azopenai-langchain.openai.azure.com/",
    azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    # api_key="sk-...", # For key-based authentication.
)

In [ ]:
agent= AssistantAgent( name = "agent", 
                      model_client=az_model_client, 
                      system_message="You are a helpful AI assistant.",
                      description="This is a test agent for Azure OpenAI."
                      
                      )



human =  UserProxyAgent(name="human",description="This is a human user.",input_func= input("Enter your message: "))


termination = TextMentionTermination(text="STOP",sources=["human"])

# Create a new team with the agents and termination condition.
team_1 = RoundRobinGroupChat(participants=[agent, human],
                           termination_condition=termination,max_turns=5)

#await team.reset()

async for message in  team_1.run_stream(task="Write a poem about the fall season in 2 lines."):
    print(message.content)
    print(message.source)
    print("-----------------------------------------------------------")

In [12]:
assistant = AssistantAgent("assistant", model_client=az_model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
# Use asyncio.run(...) when running in a script.
await Console(stream)

---------- TextMessage (user) ----------
Write a 4-line poem about the ocean.
---------- TextMessage (assistant) ----------
Waves dance beneath the endless sky,  
Whispers of secrets, where currents lie.  
In depths of blue, a world concealed,  
Nature’s bounty, forever revealed.  
TERMINATE
---------- TextMessage (user_proxy) ----------
Write 2 lines poem on colour pink
---------- TextMessage (assistant) ----------
Soft as dawn's first light, it blooms and glows,  
A gentle touch of sweetness, where love bestows.  
TERMINATE
---------- TextMessage (user_proxy) ----------
Approve
---------- TextMessage (assistant) ----------
TERMINATE
---------- TextMessage (user_proxy) ----------
APPROVE


TaskResult(messages=[TextMessage(id='dce513fd-eb06-477c-ab55-dc2a34809a9d', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 13, 3, 34, 4, 777767, tzinfo=datetime.timezone.utc), content='Write a 4-line poem about the ocean.', type='TextMessage'), TextMessage(id='060a0977-1e68-44e6-8b03-f257df0e226a', source='assistant', models_usage=RequestUsage(prompt_tokens=46, completion_tokens=41), metadata={}, created_at=datetime.datetime(2025, 7, 13, 3, 34, 16, 839395, tzinfo=datetime.timezone.utc), content='Waves dance beneath the endless sky,  \nWhispers of secrets, where currents lie.  \nIn depths of blue, a world concealed,  \nNature’s bounty, forever revealed.  \nTERMINATE', type='TextMessage'), UserInputRequestedEvent(id='bb9aa354-80a9-4bf2-a935-405cce0e87c1', source='user_proxy', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 13, 3, 34, 16, 842014, tzinfo=datetime.timezone.utc), request_id='d793e2c8-078d-49bb-ac99-313193c5f4c5', co